# Abberation mapping workflow 1: Process patches #

*25 January 2023*

*11 September 2023*

*27 September 2023*

Originally by Debayan Saha.  Jamie White modified Debayan's notebooks. 

Updating for Jamie's environment on the cluster.

Documenting installation / file locations.

Tried to update to run notebooks locally on a Mac / PC.  The psf modeling is super compute intensive, so it needs to be done on the cluster. Strategy: execute cluster commands via ssh.

**Logistics**

1. Copy the folder `patches_for_zola` to the remote using ssh (needs to be set up with ssh passkeys between local and remote (falcon). Remove the `patches_for_zola` folder from the remote before copying, if it exists. 
2. slurm calls a bashscript shell script to initialize processes on the cluster: <br> `sbatch {remote_project_path}/scripts/bashscript3.sh`
2. In turn, the bash script sets up slurm jobs on gpu nodes of the cluster and calls ImageJ to run a macro stored in the variable `remote_fiji_macro_path` in headless batch mode.
3. The ImageJ macro runs the "Calibration: PSF modeling" plugin from the with the parameters stored in the `imageJ_macro_parameters` variable.  The macro should **not** need to be modified for each experiment.
4. The ImageJ plugin models the zola_raw data for each bead in the `zola_raw` folder
5. Copy the `zola_raw` folder from the cluster to the local machine using sftp and paramiko. 

## Environment paths ##

Not necessary in local non-cuda environment; transfer these to script executed via ssh.

In [30]:
# # not necessary for running locally on a mac  
# %env PATH=/sw/ap|ps/python3/3.7.4/bin:/sw/apps/jupyter-tensorboard/0.1.6/bin:/sw/apps/hdf5/1.8.16/bin:/sw/apps/cuda/10.0.130/bin:/sw/bin:/usr/lib64/qt-3.3/bin:/sbin:/bin:/usr/sbin:/usr/bin
# %env LD_LIBRARY_PATH=/sw/apps/python3/3.7.4/lib:/sw/apps/hdf5/1.8.16/lib:/sw/apps/cuda/10.0.130/lib64:/sw/apps/cuda/10.0.130/lib:/sw/lib

In [31]:
# ! env

## Load packages ##

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append(".")
# sys.path.append('/home/dsaha/.local/lib/python3.7/site-packages/')
from main import *
import os
import shlex
import paramiko
import tifffile
import shutil
import warnings
import tqdm
from IPython.display import display, clear_output
from time import sleep

import skimage.io
import skimage.filters
import skimage.measure
import skimage.color

In [33]:
warnings.filterwarnings(action='once')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Files ###
Change the files for the experiment here.

In [34]:
# relative path to bead image stack
image_file = "data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2/beads.tif"
# image_file = "data/andrea/0p5mkBeadsDICHmirror_4_test/0p5mk.tif"

# Results file is csv of X,Y bead locations in the image stack
# by hand: maximum intensity projection of stack, then 
results_file = "Results.csv"
results_path = f"{os.path.dirname((image_file))}/{results_file}"

In [35]:
# ImageJ executable location on **cluster**
remote_fiji_path = '/home/jwhite/fiji/Fiji.app/ImageJ-linux64'
# project location on cluster
# project files are relative to this location
# and should be duplicated on the local machine
remote_project_base_path = '/home/jwhite/projects/aberration_mapping/'
# macro location: relative, for both machines
fiji_macro_dir  =  "imageJ_macros/" 
# select proper macro 
fiji_macro      = 'zola_macro_cluster_params_passed.ijm'# path to custom macros for running on cluster

In [36]:
# script for starting zola analysis on cluster
zola_script_dir = 'scripts/'
# script to use
# bashscript_fiji3.sh currently accepts paramters defined in this notebook and does not need to be edited for each analysis
zola_script     = 'bashscript_fiji3.sh'

### Parameters ###

Change the paramters for the experiment here.

In [37]:
# set new parameters
crop_shape      = [40, 40, 40]
unit            = [0.1, 0.06, 0.06]
abb_list        = [3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
thresh_bck      = 10
num_folders     = 50
thresh_dist     = 11.29999999999999999999999999999999999
lam             = 0.515

# for psf macro
pixel_size           = 100 # pixel size in nanometers
z_step               = 100 # spacing of the z-slices, in nanometers
bead_moving          = "far -> close to objective"
immersion_refractive = 1.33
numerical_aperture   = 1.2  # NA of objective used to image beads
patch_size           = 64
zernike_coefficient  = "Zernike 15 coefs"
iteration            = 100 # check deb's custom macros

In [38]:
# construct parameter string for the psf modeling ImageJ plugin 
# to be called on a gpu node of the HPC cluster by slurm
'''
run(" Calibration: PSF modeling", " run_on_gpu gain=1 pixel_size=100 z_step=50 bead_moving=[{bead_moving}] numerical_aperture=1.2 immersion_refractive=1.33 wavelength={lam} patch_size="+wid+" zernike_coefficient=[Zernike 15 coefs] iteration=100 result_calibration_file="+output);
'''
#use the gpu
imageJ_macro_parameters = f'run_on_gpu gain=1 pixel_size={pixel_size} z_step={z_step} bead_moving=[{bead_moving}] numerical_aperture={numerical_aperture} immersion_refractive={immersion_refractive} wavelength={lam} patch_size={patch_size} zernike_coefficient=[{zernike_coefficient}] iteration={iteration} result_calibration_file='
# no gpu
# imageJ_macro_parameters = f'gain=1 pixel_size={pixel_size} z_step={z_step} bead_moving=[{bead_moving}] numerical_aperture={numerical_aperture} immersion_refractive={immersion_refractive} wavelength={lam} patch_size={patch_size} zernike_coefficient=[{zernike_coefficient}] iteration={iteration} result_calibration_file='

imageJ_macro_parameters = f"\"{imageJ_macro_parameters}\""
# print(imageJ_macro_parameters)
# spaces are causing problems when passed to command line, so use "Q" as a spacer
# the > is also problematic, replace with R
imageJ_macro_parameters = imageJ_macro_parameters.replace(" ", "Q").replace(">", "R")
imageJ_macro_parameters

'"run_on_gpuQgain=1Qpixel_size=100Qz_step=100Qbead_moving=[farQ-RQcloseQtoQobjective]Qnumerical_aperture=1.2Qimmersion_refractive=1.33Qwavelength=0.515Qpatch_size=64Qzernike_coefficient=[ZernikeQ15Qcoefs]Qiteration=100Qresult_calibration_file="'

## Utility functions ##

In [39]:
def pretty_print_dict(d):
    print(json.dumps(d, indent=4))
    
def min_max_norm(arr):
    return (arr - np.min(arr))/(np.max(arr) - np.min(arr))

## Initialize paramiko for ssh and sftp connections

In [40]:
# SSH Configuration
hostname = 'falcon'
port = 22  # default SSH port, change if needed
username = 'jwhite'
# 
# local_file_path = '~/data/mydatafile.txt'
# remote_file_path = '~/data/mydatafile.txt'

# Use the private key for authentication
private_key_path = os.path.expanduser("~/.ssh/id_rsa")
private_key = paramiko.RSAKey(filename=private_key_path)

# Initialize SSH client
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())  # auto-accept the server's host key

___

## Generate additional files and parameters ##

The notebook assumes everything is set locally and tries to reproduce the local project space on the remote `falcon` using the `paramiko` python package.

## Construct local filepaths ##

From current working directory. 

In [41]:
local_base_path = os.getcwd()+'/' # everything relative to the notebook directory
local_base_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/'

In [42]:
local_fiji_macro_path = f'{local_base_path}{fiji_macro_dir}{fiji_macro}'
local_fiji_macro_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/imageJ_macros/zola_macro_cluster_params_passed.ijm'

In [43]:
# construct full path to image file and set save directory
local_img_path  = f"{local_base_path}{image_file}"
local_img_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2/beads.tif'

In [44]:
# file_path and save_dir variables for compatibility
local_data_file_path = os.path.dirname(local_img_path) +'/'
Local_save_dir  = local_data_file_path
local_data_file_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2/'

In [45]:
local_zola_script_path = f"{local_base_path}{zola_script_dir}{zola_script}"
local_zola_script_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/scripts/bashscript_fiji3.sh'

In [46]:
local_patches_path= f'{local_data_file_path}patches_for_zola/'
local_patches_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2/patches_for_zola/'

## Construct remote filepaths ##

In [47]:
remote_project_base_path

'/home/jwhite/projects/aberration_mapping/'

In [48]:
remote_image_file_path = f"{remote_project_base_path}{image_file}"
remote_image_file_path

'/home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2/beads.tif'

In [49]:
remote_fiji_macro_path = f'{remote_project_base_path}{fiji_macro_dir}{fiji_macro}'
remote_fiji_macro_path

'/home/jwhite/projects/aberration_mapping/imageJ_macros/zola_macro_cluster_params_passed.ijm'

In [50]:
# construct full path to image file and set save directory
remote_img_path  = f"{remote_project_base_path}{image_file}"
remote_img_path

'/home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2/beads.tif'

In [51]:
# file_path and save_dir variables for compatibility
remote_data_file_path = os.path.dirname(remote_img_path) +'/'
remote_save_dir  = remote_data_file_path
remote_data_file_path

'/home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2/'

In [52]:
remote_zola_script_path = f"{remote_project_base_path}{zola_script_dir}{zola_script}"
remote_zola_script_path

'/home/jwhite/projects/aberration_mapping/scripts/bashscript_fiji3.sh'

In [53]:
remote_patches_path = f'{remote_data_file_path}patches_for_zola/'
remote_patches_path

'/home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2/patches_for_zola/'

## Construct new parameters dictionary ##

In [54]:
# parameters
new_parameters = {}

new_parameters['crop_shape']      = crop_shape
new_parameters['unit']            = unit
new_parameters['abb_list']        = abb_list
new_parameters['FIJI_MACRO_PATH'] = remote_fiji_macro_path
new_parameters['thresh_bck']      = thresh_bck
new_parameters['num_folders']     = num_folders
new_parameters['thresh_dist']     = thresh_dist
new_parameters['lam']             = lam
new_parameters

{'crop_shape': [40, 40, 40],
 'unit': [0.1, 0.06, 0.06],
 'abb_list': [3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'FIJI_MACRO_PATH': '/home/jwhite/projects/aberration_mapping/imageJ_macros/zola_macro_cluster_params_passed.ijm',
 'thresh_bck': 10,
 'num_folders': 50,
 'thresh_dist': 11.3,
 'lam': 0.515}

## Process the beads stack to generate a csv file of X,Y bead locations

In [87]:
# Load the Z-stack
stack = skimage.io.imread(image_file)

# Create a maximum intensity projection
mip = np.max(stack, axis=0)

# If the image is not grayscale, convert it (though, confocal images typically are grayscale)
if len(mip.shape) > 2:
    mip = skimage.color.rgb2gray(mip)

# Apply Gaussian filter to reduce noise
smoothed_image = skimage.filters.gaussian(mip, sigma=1)

# Threshold the image to create a binary image
thresh = skimage.filters.threshold_otsu(smoothed_image)
binary_image = smoothed_image > thresh

# Label connected regions
label_image = skimage.measure.label(binary_image)

# Find the centroids of the regions
regions = skimage.measure.regionprops(label_image)

# Extract x, y positions and save to CSV
with open(results_path, 'w', newline='') as csvfile:
    bead_writer = csv.writer(csvfile)
    bead_writer.writerow(['X', 'Y'])  # Header
    for region in regions:
        y, x = region.centroid
        bead_writer.writerow([x, y])
        
# Visualization
fig, ax = plt.subplots()
ax.imshow(mip, cmap='gray')
for region in regions:
    y, x = region.centroid
    ax.plot(x, y, 'r+')
plt.show()


NameError: name 'results_path' is not defined

## Make sure everything we need is on the remote

In [55]:
# local project space will be reconstructed relative to this directory on the remote
remote_project_base_path

'/home/jwhite/projects/aberration_mapping/'

In [56]:
# compare to local project space
local_base_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/'

In [57]:
# check that the project folder is the same on both systems
local_project_check = local_base_path.split('/')[-2]
remote_project_check = remote_project_base_path.split('/')[-2]
assert local_project_check == remote_project_check,  "Local and remote base project spaces are not the same.  Are you sure about this?"

In [ ]:
# copy local image and Results.csv to remote
! scp -r "{local_save_dir}" "{remote_save_dir}"

In [79]:
# copy local files over if they are not there already

local_file_paths = [
    local_img_path,
    local_fiji_macro_path,
    local_zola_script_path
]

remote_file_paths = [
    remote_image_file_path,
    remote_fiji_macro_path,
    remote_zola_script_path
]

# open ssh connection
ssh_client.connect(hostname, port, username, pkey=private_key)

# Initialize SFTP client
sftp_client = ssh_client.open_sftp()

# check for ImageJ on remote

try:
    sftp_client.stat(remote_fiji_path)
    print(f"Using {remote_fiji_path} on the remote system {hostname}")
    print()
except FileNotFoundError:
    print(f"Cannot find {remote_fiji_path} on {hostname}.  Perhaps it is not installed?")
    print()
    
    
for local, remote in list(zip(local_file_paths, remote_file_paths)):
    # 1. Check if the remote file exists
    try:
        sftp_client.stat(remote)
        print(f"Remote file \n {remote} \nalready exists. Nothing copied.")
    except FileNotFoundError:
    # except:
     # 2. If it does not exist, copy the local file to the remote location
        sftp_client.put(local, remote)
        print(f"Local file {local} copied to {remote} on the remote server.")
    
# Close the clients
sftp_client.close()
ssh_client.close()

Using /home/jwhite/fiji/Fiji.app/ImageJ-linux64 on the remote system falcon

Remote file 
 /home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2/beads.tif 
already exists. Nothing copied.
Remote file 
 /home/jwhite/projects/aberration_mapping/imageJ_macros/zola_macro_cluster_params_passed.ijm 
already exists. Nothing copied.
Remote file 
 /home/jwhite/projects/aberration_mapping/scripts/bashscript_fiji3.sh 
already exists. Nothing copied.


## Fiji application and ImageJ macros

### Check Fiji macros ###

In [ ]:
# # check current macro directory
# fiji_macro_dir

In [ ]:
# fiji_macro_path

In [ ]:
# # check available macros
# ! ls -lga "{fiji_macro_dir}"

## Check old parameters if they exist

In [ ]:
# # look at parameters from last processing job
# old_params = setup_param_file(local_img_path)
# type(old_params)
# old_params

## Load and show the image

In [ ]:
img = tifffile.imread(local_img_path)
img_max = np.max(img,0)

%matplotlib inline
fig, axs = plt.subplots(1,1,figsize=(10,7))
plt.imshow(img_max, clim=(np.percentile(img_max,0), np.percentile(img_max,99)))
plt.title("Full image");
# None;

## Load points from csv file and save patches

In [ ]:
# from main import *
find_psf_crops(
    local_img_path, 
    crop_shape     = crop_shape, 
    num_folders    = num_folders, 
    thresh_bck     = thresh_bck, 
    threshold_dist = thresh_dist,
    unit           = unit,
    show_ignored   = False, 
    show_accepted  = False, 
    headless       = False
)

## Load patches

In [ ]:
# load the patches
patch_files = glob.glob(f"{os.path.dirname(local_img_path)}/patches_for_zola/patches*/planes_10_*.tif")
print(f"Found {len(patch_files)} patch files")
names = [os.path.splitext(os.path.basename(p))[0] for p in patch_files]
crops = {n: tifffile.imread(p) for n, p in zip(names, patch_files)}
pos = np.array([list(map(int, re.findall(r'\d+', n))) for n in names])
pos = pos[:,1:]

In [ ]:
patch_files[:2]

## Show the beads

In [ ]:
plt.imshow(img_max, clim=(np.percentile(img_max,3), np.percentile(img_max,99)))
plt.plot(pos[:,2], pos[:,1],'r.');

## Show each bead

In [ ]:
i=0
for n,pat in crops.items():
    zz,yy,xx = pat.shape
    x_arr = np.arange(-1*yy//2, yy//2)
    z_arr = np.arange(-1*zz//2, zz//2)
    fig, axs = plt.subplots(1,2,figsize=(7,3))
    print(n, np.max(pat) - np.median(pat))
    # print(shannon_entropy(pat))
    axs[0].imshow(np.max(pat,0))
    im = axs[1].imshow(np.max(pat,1))
    plt.colorbar(im)
    plt.show()
    print(pat.shape)
    i=i+1
    if i==50:
        break
None;

## Plot the signal-to-noise distribution

In [ ]:
snr = []
for key,val in crops.items():
    val = min_max_norm(val)
    snr.append(np.max(val) - np.median(val))
snr = np.array(snr)
plt.hist(snr.flatten());

## Run Zola macro via bash command on the remote

Runs via ssh on the remote cluster: CBG HPC.

Calls a macro to run an ImageJ plugin on gpu nodes of the HPC cluster in headless batch mode. 

Uses `https://github.com/imodpasteur/ZOLA-3D` plugin: put the file

`ZOLA_-0.2.8-SNAPSHOT.jar`

into the plugins directory.

Uses the ImageJ executable stored in the variable 'remote_fiji_path'

Feeds the paramters and file locations to the macro, so no need to modify the macro for each experiment.

**Logistics**

1. Copy the folder `patches_for_zola` to the remote using ssh (needs to be set up with ssh passkeys between local and remote (falcon). Remove the `patches_for_zola` folder from the remote before copying, if it exists. 
2. slurm calls a bashscript shell script to initialize processes on the cluster: <br> `sbatch {remote_project_path}/scripts/bashscript3.sh`
2. In turn, the bash script sets up slurm jobs on gpu nodes of the cluster and calls ImageJ to run a macro stored in the variable `remote_fiji_macro_path` in headless batch mode.
3. The ImageJ macro runs the "Calibration: PSF modeling" plugin from the with the parameters stored in the `imageJ_macro_parameters` variable.  The macro should **not** need to be modified for each experiment.
4. The ImageJ plugin models the zola_raw data for each bead in the `zola_raw` folder
5. Copy the `zola_raw` folder from the cluster to the local machine using sftp and paramiko.  
 

In [ ]:
# remove remote patches folder & files
! ssh falcon "rm -r {remote_patches_path}"

In [ ]:
# ! ls -lga "{local_patches_path}"

In [ ]:
! scp -r "$local_patches_path" falcon:$remote_data_file_path

### Troubleshooting / Testing ###

**Test Deb's ImageJ macro**

Note: to use the "run_on_gpu" option for the zola Fiji plugin in this line of the macro:

```python

	run(" Calibration: PSF modeling", "run_on_gpu gain=1 pixel_size=100 z_step=50 bead_moving=[far -> close to objective] numerical_aperture=1.2 immersion_refractive=1.33 wavelength=0.515 patch_size="+wid+" zernike_coefficient=[Zernike 15 coefs] iteration=100 result_calibration_file="+output);
```

you need to be logged into a gpu system on the cluster or implement the bashscript_fiji.sh (and derivatives) with sbatch on cluster (the script requests a gpu).

In [ ]:
# i = 19
# INPUT_DIR=f"{remote_save_dir}patches_for_zola/patches_{i}"
# INPUT_DIR
# remote_zola_script_path
# # remote_imageJ_macro_parameter_string = shlex.quote(f"{INPUT_DIR},{imageJ_macro_parameters}")
# # remote_imageJ_macro_parameter_string = f'param0,param1'
# remote_imageJ_macro_parameter_string =  f"{INPUT_DIR},{imageJ_macro_parameters}"
# remote_imageJ_macro_parameter_string
# # print(remote_imageJ_macro_parameter_string)
# print(f"sbatch {remote_zola_script_path} '{remote_fiji_path} {remote_fiji_macro_path} {remote_imageJ_macro_parameter_string}'")

In [ ]:
# ! scp "{local_fiji_macro_path}" falcon:"{remote_fiji_macro_path}"
# ! scp "{local_zola_script_path}" falcon:"{remote_zola_script_path}"
# 
# # ! ssh falcon sbatch {remote_zola_script_path} '{remote_fiji_path} {remote_fiji_macro_path} {remote_imageJ_macro_parameter_string}'

In [ ]:
# ! ssh falcon "squeue -u jwhite"

In [ ]:
# !ssh falcon "scancel 48960858"

In [ ]:
# be sure the std/ directory exists
# !sbatch  -J {i} -e std/{i}.err -o std/{i}.out /home/jwhite/projects/aberration_mapping/scripts/bashscript_fiji.sh {FIJI_MACRO_PATH} {INPUT_DIR}

### Run Zola analysis on cluster ###

In [ ]:
# copy fiji macro and zola bashscript to remote
! scp "{local_fiji_macro_path}" falcon:"{remote_fiji_macro_path}"
! scp "{local_zola_script_path}" falcon:"{remote_zola_script_path}"

In [ ]:
# remove existing zola folder and create a new one
remote_zola_raw_folder = f"{remote_patches_path}zola_raw/"
local_zola_raw_folder = f"{local_patches_path}zola_raw/"
# remote_zola_raw_folder
! ssh falcon "rm -r {remote_zola_raw_folder}; mkdir {remote_zola_raw_folder}" 

In [ ]:
! ssh falcon "ls -lga {remote_save_dir}"

In [ ]:
# Deleting the zola_raw folder and creating a new one
# shutil.rmtree(f"{save_dir}/patches_for_zola/zola_raw/", ignore_errors=True)
# os.makedirs(  f"{save_dir}/patches_for_zola/zola_raw/")
# 
# INPUT_DIR=f"{save_dir}/patches_for_zola/patches_{i}"

In [ ]:
# not sure why Debayan had jobs split into two groups
#
# ***** be sure the std/ directory exists *****
#
# running zola on all the patches
for i in range(35):
# for i in [28]:
# for i in range(1):
# for i in range(0,num_folders):
    INPUT_DIR=f"{remote_save_dir}patches_for_zola/patches_{i}"
    print(INPUT_DIR)
    remote_imageJ_macro_parameter_string =  f"{INPUT_DIR},{imageJ_macro_parameters}"
    # !sbatch -J {i} -e std/{i}.err -o std/{i}.out {zola_script_path} {FIJI_MACRO_PATH} {INPUT_DIR}
    ! ssh falcon sbatch {remote_zola_script_path} '{remote_fiji_path} {remote_fiji_macro_path} {remote_imageJ_macro_parameter_string}'    
    print()
#     break

for i in range(35, num_folders):
    INPUT_DIR=f"{remote_save_dir}patches_for_zola/patches_{i}"
    print(INPUT_DIR)
    remote_imageJ_macro_parameter_string =  f"{INPUT_DIR},{imageJ_macro_parameters}"
    # print(zola_script_path)
    # !sbatch -J {i} -e std/{i}.err -o std/{i}.out {zola_script_path} {FIJI_MACRO_PATH} {INPUT_DIR}
    ! ssh falcon sbatch {remote_zola_script_path} '{remote_fiji_path} {remote_fiji_macro_path} {remote_imageJ_macro_parameter_string}'
    print()

In [ ]:
! ssh falcon "squeue -u jwhite"

## Write current parameters to file

In [ ]:
local_data_file_path

In [ ]:
# Write to params file
params = {
    'crop_shape':crop_shape, 
    'unit':unit, 
    'abb_list': abb_list,  
    'COMPUTERNAME' : socket.gethostname(),
    'thresh_bck':thresh_bck, 
    'lam':lam, 
    'thresh_dist':thresh_dist,
    'fiji_macro': fiji_macro,
    'remote_project_base_path': f"{remote_project_base_path}",
    'remote_fiji_path' : f"{remote_fiji_path}",
    "zola_script" : f"{zola_script}",
    "image_file": f"{image_file}",
    "remote_data_file_path" : f"{remote_data_file_path}"
}

parameter_file = "Parameters.json"
local_param_file_path = f'{local_data_file_path}{parameter_file}'
remote_param_file_path = f"{remote_data_file_path}{parameter_file}"

write_to_param_file(local_param_file_path, params)
print(local_param_file_path)
! scp '{local_param_file_path}' falcon:{remote_param_file_path}

## Monitor progress

In [ ]:
l = !ssh falcon "squeue -u jwhite"
l = l[1:]
l

In [ ]:
# monitor progress....
l = !ssh falcon "squeue -u jwhite"
l = l[1:]
n = len(l)
while n > 0:
    clear_output(wait=False)
    l = !ssh falcon "squeue -u jwhite"
    l = l[1:]
    n=len(l)
    display(f'{n} files to go')
    sleep(1)

## Sample Fiji macro

```
/*
 * @Debayan MPI-CBG
 */

file_path = getArgument();
print("In macro");
print(file_path);
processFolder(file_path); 

// function to scan folders/subfolders/files to find files with correct suffix
function processFolder(input) {
	start = getTime();
	print(input)
	list = getFileList(input);
	list = Array.sort(list);
	print(list.length)
	for (i = 0; i < list.length; i++) {
		print(list[i]);
		if(File.isDirectory(input + File.separator + list[i])){
			l = substring(list[i], 0, lengthOf(list[i])-1);
			processFolder(input + File.separator + l + File.separator);
		}
		if(endsWith(list[i], ".tif")){
			processFile(input, list[i]);
		}
	}
	print("Totoal Time: "+(getTime()-start)/1000);   
}

function processFile(input, file) {
	print("Processing: "+ input + File.separator + file);
	if (indexOf(input, "patches") > 0) {
		if (!File.exists(File.getParent(input) + File.separator + "zola_raw")){
			File.makeDirectory(File.getParent(input) + File.separator + "zola_raw"); 
		}
	output = File.getParent(input) + File.separator + "zola_raw"+ File.separator + substring(file, 0, indexOf(file, '.tif'))+'.json';
	print(output);

	open(input + File.separator + file);
	setSlice((nSlices+1)/2); 
	makePoint(getWidth()/2, getHeight()/2);
	wid = 64;
	run(" Calibration: PSF modeling", "run_on_gpu gain=1 pixel_size=200 z_step=50 bead_moving=[far -> close to objective] numerical_aperture=1.2 immersion_refractive=1.33 wavelength=0.515 patch_size="+wid+" zernike_coefficient=[Zernike 15 coefs] iteration=20 result_calibration_file="+output);
	run("Close All");
	}
	else{print("No patches");
	}
}

```

## Sample Zola bash scripts ##



**`/projects/project-dsaha/insitu_psf/bashscript_fiji.sh`**

```
#!/bin/bash
#SBATCH -J "One"
#SBATCH -t 8:00:00
#SBATCH -n 1
#SBATCH -N 1
#SBATCH --partition=gpu
#SBATCH --gres=gpu:1
#SBATCH --mem 1000


#export OPENBLAS_NUM_THREADS=4

module unload cuda
module load cuda/10.0.130
#module load cuda/9.0.176

echo "Starting Fiji script"


FIJI_MACRO_FILEPATH=$1
PATCHES_FOLDER_PATH=$2
echo ${FIJI_MACRO_FILEPATH}
echo ${PATCHES_FOLDER_PATH}
echo "Bashscript"
/projects/project-dsaha/Fiji_jobs/Fiji.app/ImageJ-linux64 --headless -batch ${FIJI_MACRO_FILEPATH} ${PATCHES_FOLDER_PATH}

echo "Finishing Fiji script"

```

**`/home/jwhite/projects/aberration_mapping/scripts/bashscript_fiji.sh`**

```
#!/bin/bash
#SBATCH -J "One"
#SBATCH -t 8:00:00
#SBATCH -n 1
#SBATCH -N 1
#SBATCH --partition=gpu
#SBATCH --gres=gpu:1
#SBATCH --mem 1000


#export OPENBLAS_NUM_THREADS=4

module unload cuda
module load cuda/10.0.130
#module load cuda/9.0.176

echo "Starting Fiji script"


FIJI_MACRO_FILEPATH=$1
PATCHES_FOLDER_PATH=$2
echo ${FIJI_MACRO_FILEPATH}
echo ${PATCHES_FOLDER_PATH}
echo "Bashscript"
/home/jwhite/fiji/Fiji.app/ImageJ-linux64 --headless -batch ${FIJI_MACRO_FILEPATH} ${PATCHES_FOLDER_PATH}

echo "Finishing Fiji script"


```

**`/projects/project-dsaha/insitu_psf/bashscript_fiji_short.sh`**

```
#!/bin/bash
#SBATCH -J "One"
#SBATCH -t 1:00:00
#SBATCH -n 1
#SBATCH -N 1
#SBATCH --partition=short
#SBATCH --gres=gpu:1
#SBATCH --mem 5000


#export OPENBLAS_NUM_THREADS=4

module unload cuda
module load cuda/10.0.130
#module load cuda/9.0.176

echo "Starting Fiji script"


FIJI_MACRO_FILEPATH=$1
PATCHES_FOLDER_PATH=$2
echo ${FIJI_MACRO_FILEPATH}
echo ${PATCHES_FOLDER_PATH}
echo "Bashscript"
/projects/project-dsaha/Fiji_jobs/Fiji.app/ImageJ-linux64 --headless -batch ${FIJI_MACRO_FILEPATH} ${PATCHES_FOLDER_PATH}

echo "Finishing Fiji script"


```

# End #